In [15]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import layers
import os
import pandas as pd
import cv2

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
dataset_folder = "C:\\Users\\Aashish\\Desktop\\steeringmodel\\dataset"
final_images_folder_path = os.path.join(dataset_folder, 'final_images')
final_data_csv_path = os.path.join(dataset_folder, 'final_data.csv')

final_data = pd.read_csv(final_data_csv_path)

In [17]:
final_data

,timestamp,angle,throttle,brake
0,147942421588,0.001745,0.395651,0.148135
1,147942421598,0.003491,0.397726,0.148806
2,147942421608,0.006981,0.410178,0.147951
3,147942421618,0.026180,0.423972,0.148425
4,147942421628,0.031416,0.427024,0.148623
...,...,...,...,...
16103,147942655685,0.015708,0.308934,0.146639
16104,147942655695,0.019199,0.308934,0.147234
16105,147942655705,0.017453,0.308202,0.146929
16106,147942655715,0.013963,0.305272,0.146960


In [18]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16108 entries, 0 to 16107
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  16108 non-null  int64  
 1   angle      16108 non-null  float64
 2   throttle   16108 non-null  float64
 3   brake      16108 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 503.5 KB


In [19]:
final_data[final_data['timestamp'] == 147942421608]

,timestamp,angle,throttle,brake
2,147942421608,0.006981,0.410178,0.147951


In [20]:
class Data():
    def __init__(self) -> None:
        self.path = final_images_folder_path
        self.data = []
        self.data_train = []
        self.data_val = []
        self.data_test = []
    def make_training_data(self):
        for filename in os.listdir(self.path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image = cv2.imread(os.path.join(self.path,filename))
                # image = cv2.resize(image, (320,120))
                image = image / 255.0  # Normalize image data
                image_name_as_ts = os.path.splitext(filename)[0]

                # print(f"img is {image}")

                data_row = final_data[final_data['timestamp'] == int(image_name_as_ts)]
                if not data_row.empty:
                    angle = data_row['angle'].iloc[0]
                    throttle = data_row['throttle'].iloc[0]
                    brake = data_row['brake'].iloc[0]
                    self.data.append((image, angle, throttle, brake))
        
    def training_data(self):
        self.data_train = self.data[:int((len(self.data)*3)/4)]
        return self.data_train
    
         # return last 1/4 of total data
    def validation_data(self):
        self.data_val = self.data[int((len(self.data)*3)/4)+1:-1]
        return self.data_val
    
    def testing_data(self):
        self.data_test = self.data[int(len(self.data)-1):]
        return self.data_test

In [21]:
data = Data()
data.make_training_data()

data_train = data.training_data()
print(type(data_train[0][0]))

In [ ]:
inputs = keras.Input(name='input_shape', shape=(480,600,3))

# convolutional feature maps
x = layers.Conv2D(filters=24, kernel_size=(5,5), strides=(2,2), activation='relu')(inputs)
x = layers.Conv2D(filters=36, kernel_size=(5,5), strides=(2,2), activation='relu')(x)
x = layers.Conv2D(filters=48, kernel_size=(5,5), strides=(2,2), activation='relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')(x)

# flatten layer
x = layers.Flatten()(x)

# fully connected layers with dropouts for overfit protection
x = layers.Dense(units=1152, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Dense(units=100, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Dense(units=50, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)
x = layers.Dense(units=10, activation='relu')(x)
x = layers.Dropout(rate=0.1)(x)

In [ ]:
x

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dropout_3')>

In [ ]:
# derive steering angle value from single output layer by point multiplication
steering_angle = layers.Dense(units=1, activation='linear')(x)
steering_angle = layers.Lambda(lambda X: tf.multiply(tf.atan(X), 2), name='steering_angle')(steering_angle)

# derive throttle pressure value from single output layer by point multiplication
throttle_press = layers.Dense(units=1, activation='linear')(x)
throttle_press = layers.Lambda(lambda X: tf.multiply(tf.atan(X), 2), name='throttle_press')(throttle_press)

# derive brake pressure value from single output by point multiplication
brake_pressure = layers.Dense(units=1, activation='linear')(x)
brake_pressure = layers.Lambda(lambda X: tf.multiply(tf.atan(X), 2), name='brake_pressure')(brake_pressure)


In [ ]:

# build and compile model
model = keras.Model(inputs = [inputs], outputs = [steering_angle, throttle_press, brake_pressure])
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = {'steering_angle': 'mse', 'throttle_press': 'mse', 'brake_pressure': 'mse'}
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_shape (InputLayer)       [(None, 120, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 58, 158, 24)  1824        ['input_shape[0][0]']            
                                                                                                  
 conv2d_1 (Conv2D)              (None, 27, 77, 36)   21636       ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 12, 37, 48)   43248       ['conv2d_1[0][0]']           

In [ ]:
# data = Data()
# data.make_training_data()
# data_train = data.training_data()
# data_val = data.validation_data()
# data_test = data.testing_data()
# print(np.array([data[0] for data in data_train]).shape, np.array([(data[1], data[2], data[3]) for data in data_train]).shape)
# print(np.array([data[0] for data in data_val]).shape, np.array([(data[1], data[2], data[3]) for data in data_val]).shape)
# print(np.array([data[0] for data in data_test]).shape, np.array([(data[1], data[2], data[3]) for data in data_test]).shape)

In [ ]:
data = Data()
def train(model, name, data, epochs: 30, steps: 10, steps_val: 10, batch_size: 64):
    # x_train & y_train are np.array() objects with data extracted directly from the PilotData object instances
    # fit data to model for training
    data.make_training_data()
    data_train = data.training_data()
    data_test = data.validation_data()
    print(np.array([data[0] for data in data_train]).shape)
    model.fit(np.array([data[0] for data in data_train]), np.array([(data[1], data[2], data[3]) for data in data_train]), batch_size=batch_size, epochs=epochs, steps_per_epoch=steps, validation_split=0.2, validation_steps=steps_val)
    # test the model by fitting the test data
    stats = model.evaluate(np.array([data[0] for data in data_test]), np.array([(data[1], data[2], data[3]) for data in data_test]), verbose=0)
    # print the stats
    print(f'Model accuracy: {stats}')
    # input('\nPress [ENTER] to continue...')
    # save the trained model
    # model.save(f"models/{name}.h5")
    model.save(f'{name}.h5')

In [ ]:
train(model, 'jaishreeram', data, 20, 10, 10, 32)

(9596, 120, 320, 3)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [1]:
import cv2
import numpy as np
from keras.models import load_model

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Resize the image to match the expected input shape of the model
    image = cv2.resize(image, (200, 66))
    # Normalize the pixel values to be in the range [0, 1]
    image = image.astype(np.float32) / 255.0
    # Expand dimensions to create a batch of one image
    image = np.expand_dims(image, axis=0)
    # print(image)
    return image

def predict(image_paths, model_path):
    try:
        # Load the model
        model = load_model(model_path)
    except Exception as e:
        print("An unexpected error occurred:", e)
        print("Please check if the model file path is correct and the model file is not corrupted.")
        return None

    predictions = []
    for image_path in image_paths:
        # Preprocess the image
        preprocessed_image = preprocess_image(image_path)
        # Predict using the model
        prediction = model.predict(preprocessed_image)
        predictions.append(prediction)
    
    return predictions

# Example usage
test_image_paths = ["C:\\Users\\Aashish\\Desktop\\steeringmodel\\dataset\\center\\1479424259438336477.jpg"]
model_path = 'C:\\Users\\Aashish\\Desktop\\steeringmodel\\src\\jaishreeram.h5'
predictions = predict(test_image_paths, model_path)
if predictions is not None:
    print("Predictions:", predictions)


ValueError: in user code:

    File "c:\Users\Aashish\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Aashish\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Aashish\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Aashish\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Aashish\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\

In [ ]:
final_data[final_data['timestamp']==1479424201588]

,timestamp,angle,throttle,brake
0,147942421588,0.001745,0.395651,0.148135
